# Run single CBF with bias with specified dataset and parameters

#### Load libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
%matplotlib inline
import scipy.sparse as sps

In [2]:
from Data_manager.LastFMHetrec2011.LastFMHetrec2011Reader import LastFMHetrec2011Reader
import multiprocessing, traceback, os
from functools import  partial
# TODO ADDED
# import skopt
import datetime, time
# TODO /ADDED

from ParameterTuning.SearchAbstractClass import SearchInputRecommenderParameters
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.DataSplitter_leave_k_out import DataSplitter_leave_k_out

from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender


In [3]:
dataset_class = LastFMHetrec2011Reader # Movielens10MReader # CiteULike_aReader
feature_weighting = "none" # "TF-IDF" # "BM25" # 
allow_bias_ICM = False
ICM_bias = None # None or float

In [4]:
similarity_type_list = ["cosine"]

output_folder = "result_experiments/{}".format(dataset_class.DATASET_SUBFOLDER)
if allow_bias_ICM:
    output_folder += "ICM_bias/"
else:
    output_folder += "ICM_original/"
output_folder += "feature_weighting_"+feature_weighting+"/"

dataSplitter = DataSplitter_leave_k_out(dataset_class(), k_value=1, validation_set=True)
dataSplitter.load_data()

all_available_ICM_names = dataSplitter.get_loaded_ICM_names()

print("Available ICM: ", all_available_ICM_names)

ICM_name = all_available_ICM_names[0]


DataSplitter_leave_k_out: Cold users not allowed
DataSplitter_k_fold for DataReader: LastFMHetrec2011
	 Num items: 17632
	 Num users: 1883
 	 Train interactions: 89058, density: 2.68E-03
 	 Test interactions: 1883, density: 5.67E-05
	 Validation interactions: 1883, density: 5.67E-05



	 Statistics for ICM_tags: n_features 9749, feature occurrences 108437, density: 6.31E-04


DataSplitter_k_fold: Done.
Available ICM:  ['ICM_tags']


In [5]:
print("Processing ICM: '{}'".format(ICM_name))

dataset_object = dataset_class(ICM_to_load_list = [ICM_name])

dataSplitter = DataSplitter_leave_k_out(dataset_object, k_value=1, validation_set=True)
dataSplitter.load_data()

ICM_object = dataSplitter.get_ICM_from_name(ICM_name)

Processing ICM: 'ICM_tags'
DataSplitter_leave_k_out: Cold users not allowed
DataSplitter_k_fold for DataReader: LastFMHetrec2011
	 Num items: 17632
	 Num users: 1883
 	 Train interactions: 89058, density: 2.68E-03
 	 Test interactions: 1883, density: 5.67E-05
	 Validation interactions: 1883, density: 5.67E-05



	 Statistics for ICM_tags: n_features 9749, feature occurrences 108437, density: 6.31E-04


DataSplitter_k_fold: Done.


In [6]:
ICM_object
np.unique(ICM_object.data) # NB: 0 always included, cause ICM is sparse

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  58.,  59.,  60.,  62.,  63.,  64.,  65.,  67.,  68.,  73.,
        78.,  83.,  85.,  87.,  99., 108.])

In [7]:
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

In [8]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[5, 10, 15, 20, 25, 30])

In [9]:
recommender_class = ItemKNNCBFRecommender
recommender_parameters = SearchInputRecommenderParameters(
    CONSTRUCTOR_POSITIONAL_ARGS = [ICM_object, URM_train],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {
        'topK': 50,
        'shrink': 100, 
        'similarity': 'cosine',
        'normalize': True,
        'feature_weighting': feature_weighting,
        'ICM_bias': ICM_bias,
        #**similarity_args
    }
)

In [10]:
start_time = time.time()

# Construct a new recommender instance
recommender_instance = recommender_class(*recommender_parameters.CONSTRUCTOR_POSITIONAL_ARGS,
                                         **recommender_parameters.CONSTRUCTOR_KEYWORD_ARGS)

recommender_instance.fit(*recommender_parameters.FIT_POSITIONAL_ARGS,
                         **recommender_parameters.FIT_KEYWORD_ARGS)
#                          **current_fit_parameters,
#                          **self.hyperparams_single_value)

train_time = time.time() - start_time
start_time = time.time()

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 17600 ( 100 % ), 7123.94 column/sec, elapsed time 0.04 min


In [11]:
# Evaluate recommender and get results for the first cutoff
evaluation_result_dict, evaluation_result_string = evaluator_validation.evaluateRecommender(recommender_instance)
# evaluation_result_dict = evaluation_result_dict[list(evaluation_result_dict.keys())[0]]

evaluation_time = time.time() - start_time

/Users/astronely/PycharmProjects/RecSysKNNCosineBias-master/Base/Evaluation/metrics.py:623: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/astronely/PycharmProjects/RecSysKNNCosineBias-master/Base/Evaluation/metrics.py:617: RuntimeWarning: invalid value encountered in scalar divide
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 1883 ( 100.00% ) in 0.71 seconds. Users per second: 2650


In [12]:
# Evaluate recommender and get results for the first cutoff
test_result_dict, test_result_string = evaluator_test.evaluateRecommender(recommender_instance)
# test_result_dict = test_result_dict[list(test_result_dict.keys())[0]]

test_time = time.time() - start_time

EvaluatorHoldout: Processed 1883 ( 100.00% ) in 1.28 seconds. Users per second: 1476


In [13]:
print('Train time: ', train_time)
print()
print('Validation time: ', evaluation_time)
print('Validation result:', evaluation_result_string)
print('Test time: ', test_time)
print('Test result:', test_result_string)

Train time:  2.5906448364257812

Validation time:  0.7147388458251953
Validation result: CUTOFF: 10 - ROC_AUC: 0.0718711, PRECISION: 0.0113648, PRECISION_TEST_LEN: 0.1136484, RECALL: 0.1136484, RECALL_TEST_LEN: 0.1136484, MAP: 0.0471270, MRR: 0.0471270, NDCG: nan, F1: 0.0206634, HIT_RATE: 0.1136484, ARHR: 0.0471270, RMSE: 4514.2963350, NOVELTY: 0.0063937, DIVERSITY_MEAN_INTER_LIST: 0.9735641, DIVERSITY_HERFINDAHL: 0.9973047, COVERAGE_ITEM: 0.1115585, COVERAGE_USER: 1.0000000, DIVERSITY_GINI: 0.3473760, SHANNON_ENTROPY: 9.6665867, 

Test time:  1.9971048831939697
Test result: CUTOFF: 5 - ROC_AUC: 0.0430165, PRECISION: 0.0142326, PRECISION_TEST_LEN: 0.0711630, RECALL: 0.0711630, RECALL_TEST_LEN: 0.0711630, MAP: 0.0390777, MRR: 0.0390777, NDCG: nan, F1: 0.0237210, HIT_RATE: 0.0711630, ARHR: 0.0390777, RMSE: 4195.2981295, NOVELTY: 0.0031138, DIVERSITY_MEAN_INTER_LIST: 0.9819982, DIVERSITY_HERFINDAHL: 0.9962953, COVERAGE_ITEM: 0.0788339, COVERAGE_USER: 1.0000000, DIVERSITY_GINI: 0.3706666, 